In [54]:
from __future__ import print_function

import argparse
import pickle

import numpy as np
import pandas as pd
import torch.nn as nn
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import transforms, datasets


In [56]:
print(type(train_unlabeled_loader.dataset))
print(type(train_labeled_loader.dataset))

<class 'sub.subMNIST'>
<class 'sub.subMNIST'>


In [57]:
from sub import subMNIST       # testing the subclass of MNIST dataset

trainset_original = datasets.MNIST('../data', train=True, download=True,
                                  transform=transform)

Processing
Done!


In [ ]:
train_label_index = []
valid_label_index = []
for i in range(10):
    train_label_list = trainset_original.train_labels.numpy()
    label_index = np.where(train_label_list == i)[0]
    label_subindex = list(label_index[:300])
    valid_subindex = list(label_index[300: 1000 + 300])
    train_label_index += label_subindex
    valid_label_index += valid_subindex

In [ ]:
trainset_np = trainset_original.train_data.numpy()
trainset_label_np = trainset_original.train_labels.numpy()
train_data_sub = torch.from_numpy(trainset_np[train_label_index])
train_labels_sub = torch.from_numpy(trainset_label_np[train_label_index])

In [ ]:

train_unlabel_index = []
for i in range(60000):
    if i in train_label_index or i in valid_label_index:
        pass
    else:
        train_unlabel_index.append(i)

In [ ]:
trainset_np = trainset_original.train_data.numpy()
trainset_label_np = trainset_original.train_labels.numpy()
train_data_sub_unl = torch.from_numpy(trainset_np[train_unlabel_index])
train_labels_sub_unl = torch.from_numpy(trainset_label_np[train_unlabel_index])

In [ ]:

trainset_new_unl = subMNIST(root='./data', train=True, download=True, transform=transform, k=47000)
trainset_new_unl.train_data.size()


In [ ]:
pickle.dump(trainset_new_unl, open("../data/kaggle/train_unlabeled.p", "wb" ))


In [158]:
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.1307,), (0.3081,))
                                ])
trainset_labeled_import = pickle.load(open("../data/kaggle/train_labeled.p", "rb"))
trainset_unlabeled_import = pickle.load(open("../data/kaggle/train_unlabeled.p", "rb"))
validset_import = pickle.load(open("../data/kaggle/validation.p", "rb"))
train_labeled_loader = torch.utils.data.DataLoader(trainset_labeled_import, batch_size=32, shuffle=True)
train_unlabeled_loader = torch.utils.data.DataLoader(trainset_unlabeled_import, batch_size=256,
                                                     shuffle=True)

train_unlabeled_loader.dataset.train_labels = [-1 for i in range(len(train_unlabeled_loader.dataset.train_data))]
valid_loader = torch.utils.data.DataLoader(validset_import, batch_size=32, shuffle=True)

In [159]:


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d(0.3)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        return F.sigmoid(x)


In [160]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr=.01, momentum=.8)

In [182]:

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_labeled_loader):

        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)

        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Labeled Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_labeled_loader.dataset),
                       100. * batch_idx / len(train_labeled_loader), loss.data[0]))

    for batch_idx, (data,_) in enumerate(train_unlabeled_loader):
        data = Variable(data)
        optimizer.zero_grad()
        output = model(data)
        target = Variable(torch.LongTensor(output.data.max(1)[1].numpy().reshape(-1)))
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Unlabeled Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_unlabeled_loader.dataset),
                       100. * batch_idx / len(train_unlabeled_loader), loss.data[0]))


In [186]:
def validate(epoch, valid_loader):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in valid_loader:

        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.cross_entropy(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss /= len(valid_loader) # loss function already averages over batch size
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(valid_loader.dataset),
        100. * correct / len(valid_loader.dataset)))

In [188]:

for epoch in range(1, 20):
    train(epoch)
    validate(epoch, valid_loader)



Train Labeled Epoch: 1 [0/3000 (0%)]	Loss: 2.240440
Train Labeled Epoch: 1 [320/3000 (11%)]	Loss: 2.365440
Train Labeled Epoch: 1 [640/3000 (21%)]	Loss: 2.334190
Train Labeled Epoch: 1 [960/3000 (32%)]	Loss: 2.349815
Train Labeled Epoch: 1 [1280/3000 (43%)]	Loss: 2.334402
Train Labeled Epoch: 1 [1600/3000 (53%)]	Loss: 2.256057
Train Labeled Epoch: 1 [1920/3000 (64%)]	Loss: 2.302936
Train Labeled Epoch: 1 [2240/3000 (74%)]	Loss: 2.302940
Train Labeled Epoch: 1 [2560/3000 (85%)]	Loss: 2.318558
Train Labeled Epoch: 1 [2880/3000 (96%)]	Loss: 2.334189
Train Unlabeled Epoch: 1 [0/47000 (0%)]	Loss: 1.865447
Train Unlabeled Epoch: 1 [2560/47000 (5%)]	Loss: 1.865455
Train Unlabeled Epoch: 1 [5120/47000 (11%)]	Loss: 1.865590
Train Unlabeled Epoch: 1 [7680/47000 (16%)]	Loss: 1.865463
Train Unlabeled Epoch: 1 [10240/47000 (22%)]	Loss: 1.865495
Train Unlabeled Epoch: 1 [12800/47000 (27%)]	Loss: 1.865751
Train Unlabeled Epoch: 1 [15360/47000 (33%)]	Loss: 1.865446
Train Unlabeled Epoch: 1 [17920/4700

In [190]:
testset = pickle.load(open("../data/kaggle/test.p", "rb"))

test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)


In [191]:
def test(epoch, valid_loader):
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in valid_loader:

        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
#         test_loss += F.cross_entropy(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
#         correct += pred.eq(target.data).cpu().sum()

#     test_loss /= len(valid_loader) # loss function already averages over batch size
#     print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
#         test_loss, correct, len(valid_loader.dataset),
#         100. * correct / len(valid_loader.dataset)))

In [192]:

test(1, test_loader)


In [193]:

label_predict = np.array([])
model.eval()
for data, target in test_loader:
    data, target = Variable(data, volatile=True), Variable(target)
    output = model(data)
    temp = output.data.max(1)[1].numpy().reshape(-1)
    
    label_predict = np.concatenate((label_predict, temp))

In [194]:
label_predict

predict_label = pd.DataFrame(label_predict, columns=['label'], dtype=int)
predict_label.reset_index(inplace=True)
predict_label.rename(columns={'index': 'ID'}, inplace=True)

predict_label.head()

predict_label.to_csv('../data/kaggle/sample_submission.csv', index=False)